In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
from surprise import SVD
from surprise import SVDpp

os.chdir('/Users/miftahulridwan/Documents/Block 3/Master Thesis/Experiment')
tag = 'SVDpp'
training_path = os.getcwd()+'/training/'
test_path = os.getcwd()+'/test/'
result_path = os.getcwd()+'/'+tag+'/RMSE/'
dump_path = os.getcwd()+'/'+tag+'/Prediction Dump/'

factors = [10, 20, 30, 40, 50]

# This update = i
i = 53

In [2]:
# Creating Dictionary for storing the result
RMSE = dict()

# Time logging
start_update = datetime.now()

training = pd.read_csv(training_path+'train {}.csv'.format(i))
test = pd.read_csv(test_path+'test {}.csv'.format(i))
print('Processing system update {}'.format(i))
print("="*80)

# Load the dataset to Surprise environment
reader = Reader(rating_scale=(1, 5))
train_set = Dataset.load_from_df(training[['user_id', 'movie_id', 'rating']], reader).build_full_trainset()
test_set = Dataset.load_from_df(test[['user_id', 'movie_id', 'rating']], reader).build_full_trainset().build_testset()

# Training Phase
for factor in factors:
    print('Running SVDpp for factor = {}'.format(factor))
    start_training = datetime.now()
    algo_svdpp = SVDpp(n_factors = factor, n_epochs = 30, random_state = 24, verbose = False)
    algo_svdpp.fit(train_set)
    cp1 = datetime.now()
    print("Training done in {}".format(cp1-start_training))

    # Prediction Phase
    svdpp_pred = algo_svdpp.test(test_set)
    cp2 = datetime.now()
    print("Testing done in {}".format(cp2-cp1))

    result = pd.DataFrame(svdpp_pred, columns=['user_id', 'item_id', 'rating', 'pred', 'details'])
    result.to_csv(dump_path+'update {} for factor {}'.format(i, factor))

    # Computing RMSE
    MAE_result = np.mean((result.pred - result.rating)**2)
    RMSE[factor] = np.sqrt(MAE_result)
    print("RMSE = {:.2f}".format(RMSE[factor]))
    print("factor = {} done in {}".format(factor, (cp2-start_training)))
    print('*'*80)

end_update = datetime.now()
print('System Update {} done in {}'.format(i, (end_update - start_update)))
print('='*80)
print('\n')

Processing system update 53
Running SVDpp for factor = 10
Training done in 8:46:56.469115
Testing done in 0:00:34.343894
RMSE = 0.93
factor = 10 done in 8:47:30.813009
********************************************************************************
Running SVDpp for factor = 20
Training done in 12:28:57.031316
Testing done in 0:00:37.379754
RMSE = 0.93
factor = 20 done in 12:29:34.411070
********************************************************************************
Running SVDpp for factor = 30
Training done in 16:44:53.490998
Testing done in 0:00:35.809811
RMSE = 0.93
factor = 30 done in 16:45:29.300809
********************************************************************************
Running SVDpp for factor = 40
Training done in 19:07:27.330228
Testing done in 0:00:29.734990
RMSE = 0.94
factor = 40 done in 19:07:57.065218
********************************************************************************
Running SVDpp for factor = 50
Training done in 15:02:50.494470
Testing done in 0:0